In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Set dataset path
DATASET_PATH = '/kaggle/input/okradatasetfreshnessdetection'  # Replace with your dataset path
IMAGE_SIZE = (128, 128)
BATCH_SIZE = 32

# Data Preparation

# Define paths for classes
adequate_path = '/kaggle/input/okradatasetfreshnessdetection/AdequateMature'
overmature_path = '/kaggle/input/okradatasetfreshnessdetection/OverMature'



In [9]:
# Load images and labels
def load_images_and_labels(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = load_img (os.path.join(folder, filename), target_size=IMAGE_SIZE)
        if img is not None:
            images.append(img_to_array(img))
            labels.append(label)
    return np.array(images), np.array(labels)



In [10]:
# Load both classes
adequate_images, adequate_labels = load_images_and_labels(adequate_path, 0)
overmature_images, overmature_labels = load_images_and_labels(overmature_path, 1)

# Combine data
images = np.concatenate([adequate_images, overmature_images])
labels = np.concatenate([adequate_labels, overmature_labels])

# Normalize images
images = images / 255.0

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)



TypeError: load_images_and_labels() got an unexpected keyword argument 'target_size'

In [ ]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
train_generator = train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE)



In [ ]:
# Build Classification Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])



In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Train the model
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=(X_test, y_test),
    verbose=1
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Plot Training History
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.legend()

plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define a function to test and display a single image
def test_single_image(index, X_test, y_test, model):
    # Select the image and true label
    image = X_test[index]
    true_label = y_test[index]  # Since labels are 0 or 1, we directly use them
    
    # Add batch dimension for prediction
    image_batch = np.expand_dims(image, axis=0)
    
    # Predict the class probability
    predicted_prob = model.predict(image_batch)
    predicted_label = (predicted_prob >= 0.5).astype(int)  # Convert probability to binary class
    
    # Display the image
    plt.imshow(image)
    plt.title(f"True Label: {'Adequate' if true_label == 0 else 'Overmature'} | "
              f"Predicted Label: {'Adequate' if predicted_label == 0 else 'Overmature'}")
    plt.axis('off')
    plt.show()
    
    # Print the confidence score for the "Overmature" class
    print(f"Confidence Score (Overmature): {predicted_prob[0][0]:.4f}")

# Example usage: Test and display the 38th image in the test set
test_single_image(38, X_test, y_test, model)
